In [16]:
import os 
import numpy as np
def iou_2d(box1,box2):
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    area_sum = area1 + area2
    
    #計算重疊座標 設重疊box座標為 [x1,y1,x2,y2]
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    if x1 >= x2 or y1 >= y2:
        return 0
    else:
        inter_area = (x2-x1)*(y2-y1)
    return inter_area/(area_sum-inter_area)
#'outputs_f3d/data/' 效果很差
#outputs_2dgt_noaffine_val/data/ 
#'outputs_add_projection_1class_val/data/' #now only 50 epoch
# outputs_dir/outpus_1class_affine_nocrop_val_3090/data
result_path='outputs_dir/origin/data/'
gt_path='Dataset/KITTI/training/label_2/'
#Elan_3d_box/val/Elan_3d_box_230808/label_2
gt_dict={}

result_dict={}

for filename in sorted(os.listdir(result_path)):
    result_list=[]
    with open (result_path+filename) as f :
        filename=filename.replace(".txt","")
        for line in f :
            line=line.split()
            if line[0] =="Bus" or line[0] == "Truck"  :
                continue
            else:
                result_list.append({'class':line[0],'alpha':float(line[3]),'boxs':line[4:8],'group':-1,'depth':float(line[13]),'3d_attribute':line[8:15],'id':filename}) #8,9,10
    result_dict[filename]=result_list
for gt_filename in sorted(os.listdir(gt_path)):
    gt_list=[]
    with open(gt_path+gt_filename) as gt_f:
        gt_filename=gt_filename.replace(".txt","")
        for line in gt_f:
            line=line.split()
            if line[0] != "Car":
                continue
            else:
                gt_list.append({'class':line[0],'alpha':float(line[3]),'boxs':line[4:8],'group':-1,'depth':float(line[13]),'3d_attribute':line[8:15],'id':gt_filename})
        gt_dict[gt_filename]=gt_list

for key,file in gt_dict.items():
    for index,info in enumerate(file):
        file[index]['boxs']=list(map(float,file[index]['boxs']))
        file[index]['3d_attribute']=list(map(float,file[index]['3d_attribute']))
        

for key,file in result_dict.items():
    for index,info in enumerate(file):
        file[index]['boxs']=list(map(float,file[index]['boxs']))
        file[index]['3d_attribute']=list(map(float,file[index]['3d_attribute']))
count=0
for key,file in gt_dict.items():
    for index,info in enumerate(file):
        # print(file[index]['3d_attribute'][6])
        if (file[index]['3d_attribute'][6] < -np.pi):
            file[index]['3d_attribute'][6] +=2*np.pi

def matching (gt_label,result_label):
    for index,info in enumerate(gt_label):
        tempoverlap=0
        for result_label_index,result_label_info in enumerate(result_label): 
            overlap=iou_2d(gt_label[index]['boxs'],result_label[result_label_index]['boxs'])
            if overlap>tempoverlap and overlap>0.3:
                tempoverlap=overlap
                #print(second_frame_index)
                gt_label[index]['group']=result_label_index
                result_label[result_label_index]['group']=result_label_index


In [25]:
def depth_diff(gt_label,result_label,):
    temp_depth_list=[]
    temp_angle_list=[]
    temp_dim_list=[]
    for id,info in enumerate(result_label):
        for gt_id,gt_info in enumerate(gt_label):
            if (result_label[id]['group']==gt_label[gt_id]['group']) and result_label[id]['group']!=-1 and gt_label[gt_id]['group']!=-1:
                temp_depth_list.append(abs(result_label[id]['depth']-gt_label[gt_id]['depth']))
                temp_angle_list.append(abs(result_label[id]['3d_attribute'][6]-gt_label[gt_id]['3d_attribute'][6]))
                temp_dim_list.append(abs(np.array(result_label[id]['3d_attribute'][0:3])-np.array(gt_label[gt_id]['3d_attribute'][0:3])))
                # print(result_label[id]['alpha']-gt_label[gt_id]['alpha'])
                break
    return temp_depth_list,temp_angle_list,temp_dim_list
continue_depth_list=[]
continue_angle_list=[]
continue_dim_list=[]
for filename in sorted(os.listdir(result_path)):
    filename=filename.replace(".txt","")
    matching(gt_dict[filename],result_dict[filename])
    
    depth_list,angle_list,dim_list=depth_diff(gt_dict[filename],result_dict[filename])
    # print(depth_list)
    continue_depth_list+=depth_list
    continue_angle_list+=angle_list
    continue_dim_list+=dim_list

continue_dim_list=(np.array(continue_dim_list))
print(np.array(continue_depth_list).shape)
print(np.mean(continue_dim_list,axis=0))
print(np.mean(continue_angle_list))
print(np.mean(continue_depth_list))


(9340,)
[0.07561991 0.07209422 0.31591328]
0.42813276231263375
1.0019079229122054


In [26]:
print((continue_angle_list))
continue_angle_list = [x for x in continue_angle_list if x<3]

[0.030000000000000027, 0.37, 0.1100000000000001, 0.0, 0.029999999999999805, 0.010000000000000009, 0.050000000000000044, 0.11999999999999988, 0.030000000000000027, 0.0, 0.039999999999999813, 0.020000000000000018, 3.16, 0.0, 0.040000000000000036, 0.020000000000000018, 0.050000000000000044, 0.06000000000000005, 0.010000000000000009, 0.040000000000000036, 0.06000000000000005, 0.040000000000000036, 0.020000000000000018, 0.010000000000000009, 0.010000000000000009, 0.050000000000000044, 0.010000000000000009, 0.030000000000000027, 0.09000000000000008, 0.0, 0.0, 0.06000000000000005, 0.010000000000000009, 0.040000000000000036, 0.030000000000000027, 0.020000000000000018, 0.010000000000000009, 0.0, 0.040000000000000036, 0.07999999999999985, 0.010000000000000009, 0.020000000000000018, 0.050000000000000044, 0.020000000000000018, 0.040000000000000036, 0.009999999999999787, 0.15999999999999992, 0.039999999999999813, 0.040000000000000036, 0.04999999999999982, 0.030000000000000027, 0.040000000000000036,

In [27]:
print(np.mean(continue_angle_list))

0.07332904884318765
